In [1]:
import findspark
findspark.init()
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, StringType, StructType, TimestampType
import os
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

In [2]:
# JSON Schema
json_schema = StructType() \
    .add('ImpressionID', StringType()) \
    .add('UserID', StringType()) \
    .add('SessionID', StringType()) \
    .add('AdID', StringType()) \
    .add('AdCategory', StringType()) \
    .add('Timestamp', TimestampType()) \
    .add('OS', StringType()) \
    .add('Browser', StringType()) \
    .add('Location', StringType()) \
    .add('Referrer', StringType()) \
    .add('ImpressionCost', FloatType()) \
    .add('ClickID', StringType()) \
    .add('ClickPosition', StringType()) \
    .add('ClickCost', FloatType())

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .config("spark.jars", "postgresql-42.7.1.jar") \
    .appName("impression-event") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/11 21:44:15 WARN Utils: Your hostname, James-Razerblade resolves to a loopback address: 127.0.1.1; using 192.168.204.183 instead (on interface eth0)
24/06/11 21:44:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/limws/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/limws/.ivy2/cache
The jars for the packages stored in: /home/limws/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-834b3b05-985a-4255-b698-0f4f549a3cf1;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-loggi

In [4]:
impression_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "impression-event") \
  .option("startingOffsets", "latest") \
  .load()

In [5]:
impression_json_df = impression_df.selectExpr("cast(value as string) as value")
raw_df = impression_json_df.withColumn("value", from_json(impression_json_df["value"], json_schema)).select("value.*")

In [6]:
raw_df = raw_df.replace("\"NaN\"", None)

In [7]:
conn = psycopg2.connect(
    host="localhost",
    database="superset",
    user="superset",
    password="superset")

sql = """DROP TABLE IF EXISTS impression CASCADE"""
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [8]:
sql = """CREATE TABLE impression (
    impressionid TEXT,
    userid TEXT,
    sessionid TEXT,
    adid TEXT,
    adcategory TEXT,
    timestamp TIMESTAMP,
    device TEXT,
    os TEXT,
    browser TEXT,
    location TEXT,
    pageid TEXT,
    referrer TEXT,
    impressioncost DECIMAL,
    clickid TEXT NULL,
    clickposition TEXT NULL,
    clickcost DECIMAL NULL 
);"""
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [9]:
def write_to_postgresql(df,epoch_id):
    df.write \
    .format('jdbc') \
    .option("url", "jdbc:postgresql://localhost:5432/superset") \
    .option("dbtable", "impression") \
    .option("user", "superset") \
    .option("password", "superset") \
    .option("driver", "org.postgresql.Driver") \
    .mode('append') \
    .save()

In [10]:
postgresql_stream=raw_df.writeStream \
    .foreachBatch(write_to_postgresql) \
    .start() \
    .awaitTermination()

24/06/11 21:44:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0016f6a7-1ec1-4319-a178-5d39ddf18cab. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/11 21:44:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/06/11 21:44:26 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/06/11 21:44:26 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/06/11 21:44:26 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/06/11 21:44:26 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known con